In [ ]:
%%capture
!sudo sed -i.bak 's|http://archive.ubuntu.com/|http://mirror.math.princeton.edu/pub/ubuntu/|g' /etc/apt/sources.list
!sudo apt update
!sudo apt install poppler-utils


In [ ]:
# %%capture
## Installation of required libraries
!apt-get install poppler-utils
!pip install tesseract libmagic pandoc
!sudo apt-get install libleptonica-dev tesseract-ocr tesseract-ocr-dev libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn
!sudo apt-get install tesseract-ocr

In [ ]:
%pip install -q "unstructured[all-docs]" unstructured-client watermark
!pip install faiss-cpu
!pip install -q langchain langchain-community
!pip install pymupdf


In [ ]:
from unstructured.partition.pdf import partition_pdf
from collections import defaultdict
import re
import os
# from dotenv import load_dotenv
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import openai

import fitz  # PyMuPDF
import base64
import os
from collections import defaultdict


In [ ]:
# Utility Functions
def clean_text(text):
    """Cleans text by removing unwanted characters and excessive whitespace."""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,;!?-]', '', text)
    return text.strip()


In [ ]:

def process_pdf_with_unstructured(pdf_path):
    """Processes the PDF using unstructured library and organizes data."""
    elements = partition_pdf(
        filename=pdf_path,
        include_page_breaks=True,
        strategy="auto",
        infer_table_structure=True,
        extract_images_in_pdf=True,
    )
    content_by_page = defaultdict(lambda: {"text": [], "images": [], "tables": []})

    for element in elements:
        metadata = element.metadata.to_dict()
        page_number = metadata.get("page_number", "Unknown")
        element_type = getattr(element, "type", None)

        if element_type == "Text":
            content_by_page[page_number]["text"].append(clean_text(element.text or ""))
        elif element_type == "Image":
            content_by_page[page_number]["images"].append(element.to_dict())
        elif element_type == "Table":
            content_by_page[page_number]["tables"].append(element.to_dict())
        else:
            content_by_page[page_number]["text"].append(clean_text(str(element)))

    for page, content in content_by_page.items():
        content["text"] = " ".join(content["text"])

    try:
        del content_by_page['Unknown']
    except Exception:
        pass

    return content_by_page


In [ ]:
# !pip install nltk
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
res = process_pdf_with_unstructured('/content/license.pdf')

In [ ]:
!pip install ragas

In [ ]:
!pip install langchain-groq langchain lanGchain-community

In [ ]:
docs = []
for k,v in res.items():
  docs.append({k:v['text']})

In [ ]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType

kg = KnowledgeGraph()
for one in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={
                "page_content": list(one.values())[0],
                "document_metadata": str(list(one.keys())[0]),
            },
        )
    )

In [ ]:
kg

KnowledgeGraph(nodes: 10, relationships: 0)

In [ ]:
from google.colab import userdata
import os
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from langchain_groq import ChatGroq
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# llm = ChatGroq(temperature=0, groq_api_key=userdata.get('GROQ_API_KEY'), model_name="llama3-8b-8192")
# evaluator_llm = LangchainLLMWrapper(llm)



In [ ]:
# !pip install langchain-openai
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini",api_key = userdata.get('OPENAI_API_KEY'), temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)


In [ ]:
from ragas.testset.transforms import apply_transforms,Parallel
from ragas.testset.transforms.relationship_builders.traditional import RelationshipBuilder,JaccardSimilarityBuilder
from ragas.testset.transforms import (
    HeadlinesExtractor,
    HeadlineSplitter,
    KeyphrasesExtractor
)
from ragas.testset.transforms.extractors import NERExtractor

headline_extractor = HeadlinesExtractor(llm=evaluator_llm)
headline_splitter = HeadlineSplitter(min_tokens=300, max_tokens=1000)
keyphrase_extractor = KeyphrasesExtractor(
    llm=evaluator_llm, property_name="keyphrases", max_num=10
)

transforms = [
    Parallel(
      NERExtractor(llm=evaluator_llm,),
      headline_extractor,
      keyphrase_extractor
    ),
    JaccardSimilarityBuilder(property_name="entities", new_property_name="entity_jaccard_similarity")
]

apply_transforms(kg, transforms=transforms)

Applying [NERExtractor, HeadlinesExtractor, KeyphrasesExtractor]:   0%|          | 0/39 [00:00<?, ?it/s]

ERROR:ragas.testset.transforms.engine:unable to apply transformation: expected string or buffer
ERROR:ragas.testset.transforms.engine:unable to apply transformation: expected string or buffer
ERROR:ragas.testset.transforms.engine:unable to apply transformation: expected string or buffer
ERROR:ragas.testset.transforms.engine:unable to apply transformation: expected string or buffer
ERROR:ragas.testset.transforms.engine:unable to apply transformation: expected string or buffer
ERROR:ragas.testset.transforms.engine:unable to apply transformation: expected string or buffer


Applying JaccardSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:ragas.testset.transforms.engine:unable to apply transformation: Node 6814eb61-4859-41d9-9d5b-0f0d10f5f4ce or 10893b04-c824-429d-b20e-ed3fec046444 has no entities


In [ ]:
kg.relationships

[Relationship(Node(id: afbf3e) -> Node(id: eacdf3), type: child, properties: []),
 Relationship(Node(id: afbf3e) -> Node(id: 0d9e45), type: child, properties: []),
 Relationship(Node(id: afbf3e) -> Node(id: 13d4f9), type: child, properties: []),
 Relationship(Node(id: eacdf3) -> Node(id: 0d9e45), type: next, properties: []),
 Relationship(Node(id: 0d9e45) -> Node(id: 13d4f9), type: next, properties: [])]

In [ ]:
from ragas.testset.persona import Persona

persona_new_joinee = Persona(
    name="New Joinee",
    role_description="Don't know much about the company and is looking for information on how to get started.",
)
persona_manager = Persona(
    name="Manager",
    role_description="Wants to know about the different teams and how they collaborate with each other.",
)
persona_senior_manager = Persona(
    name="Senior Manager",
    role_description="Wants to know about the company vision and how it is executed.",
)

personas = [persona_new_joinee, persona_manager, persona_senior_manager]
personas

[Persona(name='New Joinee', role_description="Don't know much about the company and is looking for information on how to get started."),
 Persona(name='Manager', role_description='Wants to know about the different teams and how they collaborate with each other.'),
 Persona(name='Senior Manager', role_description='Wants to know about the company vision and how it is executed.')]

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=userdata.get('OPENAI_API_KEY')
)

In [ ]:
from ragas.testset import TestsetGenerator
from ragas.testset.graph import KnowledgeGraph
from ragas.llms import llm_factory

# Load the knowledge graph
# kg = KnowledgeGraph.load("../../../../experiments/gitlab_kg.json")
# Initialize the Generator LLM
# llm = llm_factory("gpt-4o-mini")

# Initialize the Testset Generator
testset_generator = TestsetGenerator(knowledge_graph=kg,embedding_model=embeddings, persona_list=personas, llm=evaluator_llm)
# Generate the Testset
testset = testset_generator.generate(testset_size=10)
testset

In [ ]:
testset

In [ ]:
dataset = testset.to_pandas()
dataset.rename(columns={"reference_contexts": "retrieved_contexts"}, inplace=True)
dataset.rename(columns={"question": "response"}, inplace=True)

,user_input,retrieved_contexts,reference,synthesizer_name
0,What is the significance of Dodda Kannelli in ...,[END USER LICENSE AGREEMENT PLEASE READ THIS C...,Dodda Kannelli is the registered office locati...,single_hop_specifc_query_synthesizer
1,Wot is Third Party Software in dis agreement?,[Subject to the terms and conditions of this A...,Third Party Software refers to proprietary sof...,single_hop_specifc_query_synthesizer
2,What Wipro do if Licensee no pay license fees?,[compliance with this Agreement. Any such audi...,If the Licensee fails to pay license fees cons...,single_hop_specifc_query_synthesizer
3,What Wipro do if there is a claim of infringem...,"[the Software infringes any patent, copyright,...",Wipro will have the exclusive right to defend ...,single_hop_specifc_query_synthesizer
4,What role does Wipro play in the context of da...,[Initial Term and any Renewal Terms together s...,"Wipro is identified as the data processor, whi...",single_hop_specifc_query_synthesizer
5,What Licensee must do before audit Wipro?,[verify Wipros confidentiality and security me...,Licensee shall give reasonable notice to Wipro...,single_hop_specifc_query_synthesizer
6,What is MySQL 5.7.29 in relation to the Wipro ...,[Annexure A ORDER FORM The Software being lice...,MySQL 5.7.29 is listed as a third-party softwa...,single_hop_specifc_query_synthesizer
7,What are the licensing terms associated with R...,[Annexure B 1. Red Hat Products EULA link http...,The context does not provide specific licensin...,single_hop_specifc_query_synthesizer


In [ ]:
data_list = dataset.to_dict(orient='records')

In [ ]:
import openai

# Function to get a response from OpenAI's LLM
def get_llm_response(query, retrieved_texts, model="gpt-4o-mini"):
    # Prepare the context from retrieved texts
    context = "\n\n".join(retrieved_texts)

    # Construct the prompt
    prompt = (
        f"You are an expert assistant. Use the following reference texts to answer the user's query.\n\n"
        f"### Reference Texts:\n{context}\n\n"
        f"### User Query:\n{query}\n\n"
        f"### Answer:"
    )

    try:
        # Send the query to OpenAI
        client = openai.OpenAI(api_key = userdata.get('OPENAI_API_KEY'))
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            temperature=0.7  # Adjust temperature for creativity
        )
        # Extract and return the LLM's response
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [ ]:
from ragas import SingleTurnSample, EvaluationDataset


dts = EvaluationDataset(samples=[
    SingleTurnSample(
        user_input=k['user_input'],
        retrieved_contexts = k['retrieved_contexts'],
        response=get_llm_response(k['user_input'],k['retrieved_contexts']),
        reference = k['reference'],
      ) for k in data_list]
)

In [ ]:
import pandas as pd
result = evaluate(
    dataset=dts,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)



Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
result

{'context_recall': 1.0000, 'faithfulness': 0.7798, 'factual_correctness': 0.6688}